In [1]:
import datasets
import pandas as pd
import os

In [2]:
# actionbility_mapping = {
#     "Unactionable": "1", 
#     "Borderline Actionable": "2", 
#     "Somewhat Actionable": "3", 
#     "Mostly Actionable": "4", 
#     "Highly Actionable": "5"
# }


def map_values(example):
    example["actionability"] = example["actionability"][0] if 'actionability' in example.keys() and example["actionability"] else None
    example["grounding_specificity"] = example["grounding_specificity"][0] if 'grounding_specificity' in example.keys() and example["grounding_specificity"] else None
    example['verifiability'] = example['verifiability'][0] if 'verifiability' in example.keys() and example['verifiability'] else None
    example['helpfulness'] = example['helpfulness'][0] if 'helpfulness' in example.keys() and example['helpfulness'] else None
    example['professional_tone'] = example['professional_tone'][0] if 'professional_tone' in example.keys() and example['professional_tone'] else None
    example['valid_point'] = example['valid_point'][0] if 'valid_point' in example.keys() and example['valid_point'] else None
    example['addressed_to_author'] = example['addressed_to_author'][0] if 'addressed_to_author' in example.keys() and example['addressed_to_author'] else None
    if example['addressed_to_author'] == 'authors':
        example['addressed_to_author'] = '1'
    elif example['addressed_to_author'] == 'meta_reviewer':
        example['addressed_to_author'] = '0'
    else :
        example['addressed_to_author'] = None
    return example

In [3]:
BATCH = 9

dataset_folder = f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_{BATCH}'
human_annotations = []
## get all subdirectories in the dataset folder


with pd.ExcelWriter(f'/home/abdelrahman.sadallah/mbzuai/review_rewrite/data/human_annotation_{BATCH}/human_annotation_{BATCH}.xlsx', engine="openpyxl") as writer:

    for human_data in os.listdir(dataset_folder):
        
        dataset_name = os.path.join(dataset_folder, human_data)

        ## if this is not a directory, skip
        if not os.path.isdir(dataset_name):
            continue

        ds = datasets.load_from_disk(dataset_name)
  
        column_names  = {
            'actionability.responses' : 'actionability', 
            'grounding_specificity.responses' : 'grounding_specificity',
            'verifiability.responses' : 'verifiability',
            'helpfulness.responses' : 'helpfulness',
            'professional_tone.responses' : 'professional_tone',
            'valid_point.responses' : 'valid_point',
            'addressed_to_author.responses' : 'addressed_to_author',}
        ## delete columns that are not in the dataset
        column_names = {key: value for key, value in column_names.items() if key in ds.column_names}
        ds = ds.rename_columns(column_names)
        
        selected_columns = ['id','review_point', 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness', 'professional_tone', 'valid_point','addressed_to_author']
        ## select columns of they exist
        selected_columns = [column for column in selected_columns if column in ds.column_names]
        ds = ds.select_columns(selected_columns)

        ds = ds.map(map_values)

        df = ds.to_pandas()


        df.to_excel(writer, sheet_name=human_data, index=False)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [4]:
df

,id,review_point,actionability,grounding_specificity,verifiability,helpfulness,professional_tone,valid_point,addressed_to_author
0,train_0,- The hyperlink for footnote 3 and 4 do not se...,5,5,X,5,1,0,1
1,train_1,"- I suggest to revise a bit the discussion, es...",5,5,4,4,1,0,1
2,train_2,528 - this description of the neural network i...,5,4,2,3,1,0,1
3,train_3,2) It seems like the model is limited to CTC l...,5,3,X,4,1,0,1
4,train_4,"1. In Section 3 (line 247-252), I am wondering...",5,5,5,5,1,0,1
...,...,...,...,...,...,...,...,...,...
195,train_195,"2021 Oct;27(4):710-36. Yes, the author(s) do b...",1,5,3,1,1,1,1
196,train_196,- I’m missing comparison with a NeRF-based met...,5,5,4,4,1,0,1
197,train_197,"- Line 14, 47: A brief explanation of “multi-a...",5,5,X,5,1,0,1
198,train_198,"3. p indicates the proportion of documents, I ...",5,5,X,5,1,0,1


In [5]:
for annotation in human_annotations:
    annotator